In [1]:
import os

In [4]:
%pwd

'/home/sdp/python/text_summarization_project'

In [3]:
os.chdir("/home/sdp/python/text_summarization_project")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories, get_size

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root], verbose=True)

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir], verbose=True)

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [8]:
import os
from text_summarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [9]:
# pip install ipywidgets --upgrade

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        inputs = self.tokenizer(
            example_batch["dialogue"],
            max_length=1024,
            # padding="max_length",
            truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                example_batch["summary"],
                max_length=128,
                # padding="max_length",
                truncation=True
            )

        return{
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": labels["input_ids"]
        }
    def convert(self):
        dataset_samsum= load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset" ))

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-07-21 20:52:34,045: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-21 20:52:34,127: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-21 20:52:34,129: INFO: common: created directory at: artifacts]
[2025-07-21 20:52:34,132: INFO: common: created directory at: artifacts/data_transformation]


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]